In [ ]:
using GuSTO

In [ ]:
robot = Plane()
model = Airplane(robot)
env = Forest();

In [ ]:
using MAT
vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_straight_line_sos.mat"))
Xfull,Ufull,Tsfull = vars["X"], vars["U"], vars["t_ref"];

In [ ]:
N = 41
x_init = [1,1,5,0,robot.v_hat,0,0,robot.alpha_0]
goal_box = [110 110 5; 120 120 28]; mp = 0.5*(goal_box[1,:]+goal_box[2,:])
x_goal = [mp[1],mp[2],mp[3],0,robot.v_hat,0,0,robot.alpha_0]
tf_guess = Tsfull[end] # 32.3

full_idx = [round(Int,i) for i in collect(linspace(1,length(Tsfull),N))];

In [ ]:
# init_traj_straightline
X0 = repmat(x_init, 1, N)
for k in 1:3
    X0[k,:] = collect(linspace(x_init[k],x_goal[k],N))
end

# init_traj_line_sos
vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_straight_line_sos.mat"))
X0 = vars["X"]
U0 = vars["U"]
Nidx = size(X0,2)
idx = [round(Int,i) for i in collect(linspace(1,Nidx,N))]
X0 = X0[:,idx]
U0 = U0[:,idx[1:end-1]]
x_goal = X0[:,end]

# init_traj_feasible_sos
vars = matread(joinpath(Pkg.dir("GuSTO"), "src", "dynamics", "init_feasible_sos.mat"))
X0 = vars["X"]
U0 = vars["U"]
Nidx = size(X0,2)
idx = [round(Int,i) for i in collect(linspace(1,Nidx,N))]
X0 = X0[:,idx]
U0 = U0[:,idx[1:end-1]]
x_goal = X0[:,end];

PD = ProblemDefinition(robot, model, env, x_init, x_goal);

In [ ]:
TOPgusto = TrajectoryOptimizationProblem(PD, N, tf_guess, fixed_final_time=true)
TOSgusto = TrajectoryOptimizationSolution(TOPgusto)
solve_SCP!(TOSgusto, TOPgusto, solve_gusto_cvx!, init_traj_feasible_sos, "Gurobi", OutputFlag=0);

In [ ]:
@show TOSgusto.SCPS.successful
@show TOSgusto.SCPS.converged
@show TOSgusto.SCPS.iterations
@show TOSgusto.SCPS.total_time
@show TOSgusto.SCPS.accept_solution
@show TOSgusto.SCPS.prob_status
@show TOSgusto.SCPS.convergence_measure
@show TOSgusto.SCPS.param.alg.omega_vec
@show TOSgusto.SCPS.param.alg.Delta_vec
;

In [ ]:
PyPlot.figure()
PyPlot.plot3D(TOSgusto.traj.X[1,:],TOSgusto.traj.X[2,:],TOSgusto.traj.X[3,:], "g-.")
PyPlot.plot3D(X0[1,:], X0[2,:], X0[3,:], "k--")